In [ ]:
# Analyzer configuration
dataset_path = '/gdrive/My Drive/vaico-local/datasets/baluarte/missing-from-vrap'
mode = 'predict'

In [ ]:
# Prediction model paths
model_root = '/gdrive/My Drive/vaico-local/SURA/modelos/' 
workers_path = model_root + 'personas_yolov4-20201113.tflite'
edges_path = model_root + 'bordes_Unet-20201113.tflite'
helmet_path = model_root + 'cascos_resnet50-20201023.ml'
harness_path = model_root + 'arnes_resnet50-20201113.ml'
# Obstacles
balde_carreta_path = model_root + 'balde_carretilla_yolov4-20201023.tflite'
tabla_path = model_root + 'tablas_yolov4-20201020.tflite' 

In [ ]:
repo_path = '/gdrive/My\ Drive/vaico-local/' +'/Repos/'
mlgeometry = repo_path + 'MLgeometry-0.1.9.tar.gz'
mlevaluator = repo_path + 'MLevaluator-0.0.1.tar.gz'
mlinference = repo_path + 'mlinference-0.0.1.tar.gz'
mldrawer = repo_path + 'MLdrawer-0.0.1.tar.gz'
mlanalyzer = repo_path + 'mlanalyzer-0.0.1.tar.gz'

In [ ]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [ ]:
!pip install $mlgeometry $mlinference $mlanalyzer

Processing /gdrive/My Drive/vaico-local/Repos/MLgeometry-0.1.9.tar.gz
Processing /gdrive/My Drive/vaico-local/Repos/mlinference-0.0.1.tar.gz
Processing /gdrive/My Drive/vaico-local/Repos/mlanalyzer-0.0.1.tar.gz
     |████████████████████████████████| 516.2MB 31kB/s 
     |████████████████████████████████| 378kB 45.8MB/s 
     |████████████████████████████████| 460kB 42.9MB/s 
     |████████████████████████████████| 3.0MB 42.3MB/s 
     |████████████████████████████████| 51kB 2.7MB/s 
  Created wheel for MLgeometry: filename=MLgeometry-0.1.9-cp36-none-any.whl size=10794 sha256=b17665e8b00769499fd8fc6a01eef71f0ba93b92923b76b3e58796bbfdf25f35
  Stored in directory: /root/.cache/pip/wheels/42/d1/da/3f8ae62f7d3936cc0d45dd8c36d7d912e2505002cb30f48991
  Created wheel for mlinference: filename=mlinference-0.0.1-cp36-none-any.whl size=74672 sha256=213ebe8db09574b083a1c4a7af974e504746592a4a5d371ca06c6172a4516108
  Stored in directory: /root/.cache/pip/wheels/47/0d/c6/5ae216606abe04888445fa3e1bf3

In [ ]:
import logging
from MLinference.architectures import Yolo4, KerasClassifiers, UNet, MaskDistance, ArUco
from MLinference.strategies import Posterior, Multi, Cascade

Using TensorFlow backend.


In [ ]:
logging.basicConfig(level=logging.DEBUG)

# Create SURA models
worker_kwargs = {
    'score_threshold': 0.3,
    'overlapThresh': 0.2,
    'input_size': 608,
    'backend': 'tf', 
    'labels': {0:'persona'},
}
worker_detector = Yolo4.load(workers_path, **worker_kwargs)

helmet_classifier = KerasClassifiers.load(helmet_path, feature_extractor={'model':'resnet50_imageAI'}, pred_threshold=0.58)
harness_classifier = KerasClassifiers.load(harness_path, feature_extractor={'model':'resnet50_imageAI'}, pred_threshold=0.58)

indentifier = ArUco(None)

# Obstacles Models
balde_carreta_kwargs = {
    'input_size': 608, 
    'backend': 'tf', 
    'labels': {0:'balde', 1:'carretilla'},
    'score_threshold': 0.3,
    'overlapThresh': 0.2,
}
balde_carreta_detector = Yolo4.load(balde_carreta_path,**balde_carreta_kwargs)

tabla_kwargs = {
    'input_size': 608, 
    'backend': 'tf', 
    'labels': {0:'tabla'},
    'score_threshold': 0.3,
    'overlapThresh': 0.2,
}
tabla_detector = Yolo4.load(tabla_path,**tabla_kwargs)



worker_cascade = Cascade(
    main_model={'model':worker_detector},
    sub_models={
        'persona': [
                    {
                        'model': helmet_classifier,
                        'weights': (0, -0.05, 1.2, 1.1),
                        'conditions': ['center_x', 'center_y']
                     },
                    {'model': harness_classifier},
                    {'model': indentifier}
                    ]})

edges_mask = UNet.load(edges_path, labels={0:'borde'})

on_edge_kwargs = {
    'interest_labels':{
        'persona':{
            'obj_scale': 1.12,
            'pred_threshold': 1
        },'carretilla':{
            'obj_scale': 1.12,
            'pred_threshold': 1,
            'scale_reference': 'persona'
        },'balde':{
            'obj_scale': 1.12,
            'pred_threshold': 1,
            'scale_reference': 'persona'
        },'tabla':{
            'obj_scale': 1.12,
            'pred_threshold': 1,
            'scale_reference': 'persona'
        },'tubo':{
            'obj_scale': 1.12,
            'pred_threshold': 1,
            'scale_reference': 'persona'
        }
    },
    'mask_label': 'borde',
    'labels': {0:'cerca de borde', 1:'lejos de borde'},
    'keep_mask': False,
}
on_edge = MaskDistance(None, **on_edge_kwargs)

main_models = Multi(models=[worker_cascade, edges_mask, balde_carreta_detector, tabla_detector])

model = Posterior(models=[main_models, on_edge])

INFO:MLcommon.InferenceModel:Loading model from: /gdrive/My Drive/vaico-local/SURA/modelos/personas_yolov4-20201113.tflite
INFO:MLinference.architectures.Yolo4:Using tf backend
INFO:MLinference.architectures.Yolo4:Loaded model from: /gdrive/My Drive/vaico-local/SURA/modelos/personas_yolov4-20201113.tflite
INFO:MLcommon.AbcModel:Loading model from: /gdrive/My Drive/vaico-local/SURA/modelos/cascos_resnet50-20201023.ml
INFO:MLinference.architectures.kerasClassifiers.KerasClassifiers:Loading architecture: resnet50_imageAI
INFO:MLcommon.AbcModel:Loading weights from object
INFO:MLcommon.AbcModel:Loading model from: /gdrive/My Drive/vaico-local/SURA/modelos/arnes_resnet50-20201113.ml
INFO:MLinference.architectures.kerasClassifiers.KerasClassifiers:Loading architecture: resnet50_imageAI
INFO:MLcommon.AbcModel:Loading weights from object
INFO:MLinference.architectures.ArUco:Using {10} AR dictionary
INFO:MLinference.architectures.ArUco:Loaded model with: Default Ids
INFO:MLcommon.InferenceModel

In [ ]:
from MLanalyzer.analyzer import analyzer

In [ ]:
def saving_condition(obj):
    if obj.label == 'borde':
        return False
    return True

In [ ]:
# Analyze dataset
analyzer(dataset_path, model=model, mode=mode, saving_condition=saving_condition )

Running Analyzer. Mode: predict
Making predictions on /gdrive/My Drive/vaico-local/datasets/baluarte/missing-from-vrap
 - Creating Results folder...


  0%|          | 0/92 [00:00<?, ?it/s]

 - Predicting images from:/gdrive/My Drive/vaico-local/datasets/baluarte/missing-from-vrap
 - Saving predictions in in /gdrive/My Drive/vaico-local/datasets/baluarte/missing-from-vrap/results/predictions.json


Streaming output truncated to the last 5000 lines.
DEBUG:MLinference.strategies.Cascade:Predicting region with: <MLinference.architectures.ArUco.ArUco object at 0x7f6f760a24a8>
DEBUG:MLinference.strategies.Cascade:Predicting region with: KerasClassifiers({'_defaults': {'architecture': 'KerasClassifiers', 'dropout': 0.001, 'feature_extractor': {'model': 'resnet50_imageAI'}, 'include_top': True, ...}, 'architecture': 'KerasClassifiers', 'dropout': 0.001, 'feature_extractor': {'model': 'resnet50_imageAI'}, ...})
DEBUG:MLinference.strategies.Cascade:Predicting region with: KerasClassifiers({'_defaults': {'architecture': 'KerasClassifiers', 'dropout': 0.001, 'feature_extractor': {'model': 'resnet50_imageAI'}, 'include_top': True, ...}, 'architecture': 'KerasClassifiers', 'dropout': 0.001, 'feature_extractor': {'model': 'resnet50_imageAI'}, ...})
DEBUG:MLinference.strategies.Cascade:Predicting region with: <MLinference.architectures.ArUco.ArUco object at 0x7f6f760a24a8>
DEBUG:MLinference.str

Error: OpenCV(4.1.2) /io/opencv/modules/imgproc/src/color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cvtColor'
 on frame:/gdrive/My Drive/vaico-local/datasets/baluarte/missing-from-vrap/results
Done
